In [7]:
## Import Self Defined Library Functions

import load 
import custom_config 
import activations
import utils 
import debug

## Import Library Function

import numpy as np
#import theano
# from theano import function, config, shared, tensor
import time
import sys
sys.path
sys.path.append('/Users/achanish/anaconda/lib/python2.7/site-packages')

import mxnet as mx

## Load Data ##

trX, teX, trY, teY = load.mnist(ntrain=1000, ntest=100)
print ("Training data: ", trX.shape, trY.shape)
print ("Test data: ", teX.shape, teY.shape)
print debug.print_setting()

('Training data: ', (1000, 784), (1000, 10))
('Test data: ', (100, 784), (100, 10))
Running network:  svdMLP
Input dimension:  784
Output dimension:  10
Hidden layers:  10
Hidden dimension:  128
Number of reflection vectors:  16
Singualr margin:  0.1
Hidden-to-hidden activation function:  leaky_relu
Output activation function:  softmax
Instance per batch:  1000
Validation interval:  100
Number of epoch:  10
Learning rate:  0.0001
Droput rate:  0.1
None


In [15]:

class HiddenLayer(object):
    def __init__(self, rng, input, n_in, n_out, predict_input = None, W=None, b=None,
                 activation=activations.hyperbolic_tangent, dropout_rate = 0, nametag=''):
        """
        Typical hidden layer of a MLP: units are fully-connected and have
        sigmoidal activation function. Weight matrix W is of shape (n_in,n_out)
        and the bias vector b is of shape (n_out,).

        NOTE : The nonlinearity used here is tanh

        Hidden unit activation is given by: tanh(dot(input,W) + b)

        :type rng: numpy.random.RandomState
        :param rng: a random number generator used to initialize weights

        :type n_in: int
        :param n_in: dimensionality of input

        :type n_out: int
        :param n_out: number of hidden units

        :type activation: theano.Op or function
        :param activation: Non linearity to be applied in the hidden
                           layer
        """
        if predict_input is None:
            predict_input = input
        self.input = input
        self.predict_input = predict_input
        # end-snippet-1

        # `W` is initialized with `W_values` which is uniformely sampled
        # from sqrt(-6./(n_in+n_hidden)) and sqrt(6./(n_in+n_hidden))
        # for tanh activation function
        # the output of uniform if converted using asarray to dtype
        # theano.config.floatX so that the code is runable on GPU
        # Note : optimal initialization of weights is dependent on the
        #        activation function used (among other things).
        #        For example, results presented in [Xavier10] suggest that you
        #        should use 4 times larger initial weights for sigmoid
        #        compared to tanh
        #        We have no info for other function, so we use the same as
        #        tanh.
        if W is None:
            W_values = np.asarray(
                rng.uniform(
                    low=-np.sqrt(6. / (n_in + n_out)),
                    high=np.sqrt(6. / (n_in + n_out)),
                    size=(n_in, n_out)
                ),
                
                dtype=theano.config.floatX
            )
            if activation == theano.tensor.nnet.sigmoid:
                W_values *= 4

            W = theano.shared(value=W_values, name='W'+nametag, borrow=True)

        if b is None:
            b_values = np.zeros((n_out,), dtype=theano.config.floatX)
            b = theano.shared(value=b_values, name='b'+nametag, borrow=True)

        self.W = W
        self.b = b

        output = activation(T.dot(input, self.W) + self.b)
        predict_output = activation(T.dot(predict_input, self.W) + self.b)
        if dropout_rate > 0:
            #np.random.seed(int(time.time()))
            #mask = np.random.binomial(np.ones(n_out, dtype=int),1.0-dropout_rate)
            srng = RandomStreams(rng.randint(999999))
            mask = srng.binomial(n=1, p=1.0-dropout_rate, size=output.shape, dtype=theano.config.floatX)
            self.output = output * mask
            self.predict_output = (1.0-dropout_rate) * predict_output
        else:
            self.output = output
            self.predict_output = predict_output
        # parameters of the model
        self.params = [self.W, self.b]